<a href="https://colab.research.google.com/github/giovanaliao/Computacao_visual/blob/main/B2_image_classification_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Deep Learning - Image Classification, Pré-Trained Model**
---

Seguindo o modelo apresentado em aula crie com um **Modelo Pré-Treinado** um classificador de imagens (ver as opções de imagens em *Exercício Selecionado*, abaixo). Adcionalmente ao modelo apresentado você deve incluir:

1. **Modelo e Treinamento**: parâmetros (mais que um) de *Early-Stopping*, e parâmetros e/ou camadas de *DropOut* para reduzir o overfitting.
2. **Data augumentation**: Empregue ao menos 2 funções de **pré-processamento Keras (2)** e ao menos 2 técnicas de **métodos tf.image (2)**. Outras técnicas mais avançadas são opcionais.
3. **Compare**. Compare e discuta os resultados quando comparado ao modelo anterior com o TensorFlow.





In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '1115665, Adriana Fujita' #@param {type:"string"}
Aluno2 = '1115677, Daniel Henrique' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



### **Escolha a opção de Exercício Selecionado**

Faça a seleção aqui: https://docs.google.com/spreadsheets/d/1Y2n0JE94CNhlQzhJudVMEvTYRE5RnuSa/edit?usp=sharing&ouid=102152623479619184236&rtpof=true&sd=true

e informe:

* **Dados**:
* **Classes**:
* **Modelo Pré Treinado**:


# **Resumo**

*Apresente um parágrafo resumindo o que é feito neste trabalho*.

# **Descrição da Solução**

*Discuta aqui os principais aspectos da solução desenvolvida (um parágrafo)*.



# **Implementação**

*Aqui o código da sua solução. Você deve explicar em linhas gerais o código.*

> ***ATENÇÃO: O código deve ser 100% executável no Colab.***



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Carregamento e filtragem
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
classes_to_keep = [0, 2]
train_filter = np.isin(y_train, classes_to_keep)
test_filter = np.isin(y_test, classes_to_keep)
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]
y_train = (y_train == 2).astype(np.int32)
y_test = (y_test == 2).astype(np.int32)

# Pré-processamento
x_train = tf.image.resize(tf.expand_dims(x_train, -1), [96, 96]) / 255.0
x_test = tf.image.resize(tf.expand_dims(x_test, -1), [96, 96]) / 255.0
x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)

# Data augmentation Keras (2 funções)
datagen = ImageDataGenerator(
    zoom_range=0.2,
    rotation_range=15,  # leve aumento na rotação
    horizontal_flip=True,
    preprocessing_function=lambda img: tf.image.random_contrast(img, 0.8, 1.2)
)

# Data augmentation tf.image (2 funções)
def augment_tf(img, label):
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_flip_left_right(img)
    return img, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(augment_tf).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

# Modelo com ResNet50 aprimorado
base_model = applications.ResNet50(include_top=False, input_shape=(96, 96, 3), weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping com parâmetros aprimorados
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    min_delta=0.001,
    mode='min',
    restore_best_weights=True,
    verbose=1
)

# Treinamento
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    epochs=5,
    validation_data=val_ds,
    callbacks=[early_stop]
)


# **Resultados**

*Discuta os resultados comparando com os do modelo anterior empregando uma rede do "zero".*



# **Referências**

Este é um item obrigatório. Inclua aqui o as referências, fontes, ou bibliografia e sites/bibliotecas empregadas em formato ABNT.

---

In [ ]:
#@title **Avaliação**
Resumo = 10 #@param {type:"slider", min:0, max:10, step:1}

Descricao_solucao = 7 #@param {type:"slider", min:0, max:10, step:1}

Implementacao = 9 #@param {type:"slider", min:0, max:10, step:1}

Resultados = 6 #@param {type:"slider", min:0, max:10, step:1}

Referencias = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**
nota = Resumo + Descricao_solucao + Implementacao + Resultados + Referencias + Geral

nota = nota / 6

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.7



,tia,nome,nota
0,1115665,ADRIANA FUJITA,7.7
1,1115677,DANIEL HENRIQUE,7.7
